[For firebase realtime database ](https://console.firebase.google.com/) <br>
paste the key in a text file key.txt in ./hugging_face/database folder

In [ ]:
# @title ##**Install module** { display-mode: "form" }
!pip install ffmpeg-python==0.2.0
!pip install pip install youtube_dl
!pip install transformers
!apt install ffmpeg
!pip install spleeter
!pip install pydub
!pip install SpeechRecognition
!pip install googletrans==3.1.0a0
!pip install gTTS
!pip install db-sqlite3
!pip install pyrebase

In [ ]:
# @title ##**Import packeges** { display-mode: "form" }
from google.colab import drive
from IPython.display import clear_output
from google.colab import files
import os
import scipy
import subprocess
import requests
import glob
from pydub import AudioSegment 
from IPython.display import Audio
import shutil
import speech_recognition as srr
from googletrans import Translator
from gtts import gTTS 
from datetime import datetime , date
from collections import OrderedDict
import pyrebase


In [ ]:
# @title ##**Hugging face AI** { display-mode: "form" }
from transformers import pipeline
nlp = pipeline("question-answering")

In [ ]:
# @title ##**Required Function** { display-mode: "form" }
#sound recorder start
"""
copied
https://gist.github.com/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

#sound recorder end


# Define some helper functions for downloading pretrained model
# taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
#Download from drive start
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

#Download from drive end

# To delete file
def delete():
  for i in os.listdir():
    try:
      os.remove(i)
    except:
      pass

# Get audio_file list
def get_audio_file():
  audio_types = ["*.wav", "*.mp3"]
  audio_list = []
  for type in audio_types:
      this_type_files = glob.glob(type)
      audio_list += this_type_files
  return audio_list

#extract text from audio
def extract_text(filess):
  long_text=""
  r = srr.Recognizer()
  for i in filess:
    with srr.AudioFile(i) as source:
      # print("scan file:"+i)
      audio_data = r.record(source)
      try:
        text = r.recognize_google(audio_data,language=input_language)
        long_text+=text+" "
      except Exception as e:
        print(f"can't scan :" +i+f" empty or file_size big and the error {str(e)}")
        continue
  return long_text

#extract text from audio
def extract_question(question_path):
  long_text=""
  r = srr.Recognizer()
  with srr.AudioFile(question_path) as source:
    # print("scan file:"+i)
    audio_data = r.record(source)
    try:
      text = r.recognize_google(audio_data,language=input_language)
      long_text+=text+" "
    except Exception as e:
      print(f"can't scan :" +i+f" empty or file_size big and the error {str(e)}")
  return long_text


def input_traslator(text,language):
  # translator = Translator()
  translator = Translator(service_urls=['translate.googleapis.com','translate.google.com','translate.google.co.kr'])
  translations=translator.translate(text,src=language, dest="en")
  translate_text=translations.text
  return translate_text
def output_traslator(text,language):
  # translator = Translator()
  translator = Translator(service_urls=['translate.googleapis.com','translate.google.com','translate.google.co.kr'])
  translations=translator.translate(text,src="en", dest=language)
  translate_text=translations.text
  return translate_text  



def audio_fun(story,f_name):
  tts = gTTS(story,lang=input_language,slow = False) #Provide the string to convert to speech
  save_file=f_name+".mp3"
  tts.save(save_file) #save the string converted to speech as a .wav file
  sound_file = save_file
  Audio(sound_file, autoplay=True) 

#api for firebase
def read_key():  
  f = open("./database/key.txt", "r")
  raw_firebaseConfig=f.read()
  firebaseConfig=raw_firebaseConfig.replace(" ","").replace("\n","").split(",")
  firebaseConfig_dic={}
  for i in firebaseConfig:
    # print(i.split(":"))
    temp=i.split(":")
    if temp[0]=="apiKey":
      firebaseConfig_dic[temp[0]]=temp[1].replace('"',"")
    elif temp[0]=="authDomain":
      firebaseConfig_dic[temp[0]]=temp[1].replace('"',"")
    elif temp[0]=="databaseURL":
      key=""
      for i in range(1,len(temp)):
        key+=f"{temp[i]}:"
      key=key[:-1].replace('"','')
      firebaseConfig_dic[temp[0]]=key
    elif temp[0]=="projectId":
      firebaseConfig_dic[temp[0]]=temp[1].replace('"',"")
    elif temp[0]=="storageBucket":
      firebaseConfig_dic[temp[0]]=temp[1].replace('"',"")
    elif temp[0]=="messagingSenderId":
      firebaseConfig_dic[temp[0]]=temp[1].replace('"',"")
    elif temp[0]=="appId":
      key=""
      for i in range(1,len(temp)):
        key+=f"{temp[i]}:"
      key=key[:-1].replace('"','')
      firebaseConfig_dic[temp[0]]=key
    elif temp[0]=="measurementId":
      firebaseConfig_dic[temp[0]]=temp[1].replace('"',"")    
  return firebaseConfig_dic        

# firebase realtime database
def database(text):  
  firebaseConfig_dic=read_key()
  firebase = pyrebase.initialize_app(firebaseConfig_dic)
  db = firebase.database()
  try:
    data_count=len(db.get().val())
  except:
    data_count=0
  # print(data_count)
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")
  today = str(date.today())
  data = {"date": today, "time": current_time, "context": text }
  db.child(str(data_count)).set(data)


#read data from database
def read_database():  
  firebaseConfig_dic=read_key()
  firebase = pyrebase.initialize_app(firebaseConfig_dic)
  db = firebase.database()
  data_count=len(db.get().val())
  get_data=db.get().val()
  old_text=""
  for i in get_data:
    old_text+=" "+i["context"]
    # print(i["context"])

  old_text=old_text.replace("  "," ")
  return old_text

# reset database
def format_database():
  firebaseConfig_dic=read_key()
  firebase = pyrebase.initialize_app(firebaseConfig_dic)
  db = firebase.database()
  try:
    data_count=len(db.get().val())
  except:
    print("Database is empty")
  for i in range(0,data_count+1):
    try:
      db.child(str(i)).remove()
    except:
      pass

In [ ]:
# @title ##**Storage type** { display-mode: "form" }
%cd /content/
clear_output()
storage="google drive"#@param["temporary","google drive"]
if storage=="google drive":
  drive.mount('/content/gdrive')
  clear_output()
  os.chdir("/content/gdrive/MyDrive")
  print("Using drive as a storage")
elif storage=="temporary":
  os.chdir("/content/")
  print("Using colab as a temporary storage")  
folder_names=["input_data","database"]

if storage=="temporary":
  try:
    os.mkdir("hugging_face")
  except:
    pass
  os.chdir("./hugging_face")
  for i in folder_names:
    try:
      os.mkdir(i)
    except:
      pass

elif storage=="google drive":
  try:
    os.mkdir("hugging_face")
  except:
    pass
  os.chdir("./hugging_face")
  for i in folder_names:
    try:
      os.mkdir(i)
    except:
      pass
root=os.getcwd()

In [ ]:
# @title ##**Select your option** { display-mode: "form" }
input_type = "record_audio"#@param["record_audio","text","youtube_video","download_audio","upload_audio"]
input_language = "en" #@param["en","bn","hi"]
audio_extenstion = ".wav" #@param[".wav",".mp3"]




#delete all files in current directory
delete()

#save audio in input_data folder
os.chdir("./input_data")
delete()
#for recording
if input_type == "record_audio":
  audio, sr = get_audio()
  name="audio"
  name+=audio_extenstion
  scipy.io.wavfile.write(name, sr, audio)

#for normal text
elif input_type == "text":
  your_context=input("Paste or type the Context : ")

#for youtube video
elif input_type == "youtube_video" :
  enter_youtube_link=input("Paste the youtube link : ")
  youtube_id=enter_youtube_link.split("/")
  new_link="https://www.youtube.com/watch?v="+youtube_id[-1]                                                            
  # dummy='youtube-dl -f "bestaudio/best" -ciw -o "%(title)s.%(ext)s" -v --extract-audio --audio-quality 0 --audio-format wav  '
  dummy=f'youtube-dl {new_link} -x --audio-format {audio_extenstion.split(".")[-1]} --output "audio.%(ext)s"'
  # dummy=dummy+new_link
  with open("download_audio.sh","w") as f:
    f.write(dummy)
  !chmod 777 download_audio.sh  
  subprocess.call("./download_audio.sh", shell=True)


#for download_audio from a website or drive
elif input_type=="download_audio":  
  paste_download_link =input("Enter Download link : ")
  if "drive" in paste_download_link:
    file_id = paste_download_link.split('/')[-2]
    destination = f"audio{audio_extenstion}"
    download_file_from_google_drive(file_id, destination)
    
  else:
    try:
      urllib.request.urlretrieve(paste_download_link, f'audio{audio_extenstion}')
    except:
      pass

#for upload a audio file from a local folder
elif input_type == "upload_audio":
  uploaded = files.upload()
  clear_output()
  try:
    os.rename(get_audio_file()[0],f"audio{audio_extenstion}")
  except:
    pass
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

try:
  var=os.system(f"spleeter separate -o output/ ./input_data/audio{audio_extenstion}")
except:
  pass
if var==0:
  print("successfully spleet")
else:
  print("failed spleet ")

try:
  os.chdir(root)
except:
  pass
try:  
  shutil.copy("./output/audio/vocals.wav","./input_data")
except:
  pass
os.chdir("./input_data")
filess = []
if input_type != "text":
  vocal_file="vocals.wav"
  audio = AudioSegment.from_file(vocal_file)
  duartion=audio.duration_seconds
  if int(duartion)>20:
    command = "ffmpeg -i '"+vocal_file+"' -f segment -segment_time 20 -c copy %03d.wav"
    var1=subprocess.call(command, shell=True)
    if var1 == 0:
      print("Successfully divided audio")
    else:
      print("Failed to divided audio")
    for filename in os.listdir():
        if filename.endswith(".wav"):
            filess.append(filename)
    filess.remove(f"vocals.wav")
    filess.sort()
    text=extract_text(filess)
  else:
    text=extract_question(vocal_file)
elif input_type == "text":
  text=your_context  
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)  
!rm -rf ./output
!rm -rf ./pretrained_models

In [ ]:
# @title ##**Your voice or youtube audio** { display-mode: "form" }

Audio(f'./input_data/audio{audio_extenstion}')

In [ ]:
# @title ##**Given data** { display-mode: "form" }
text

In [ ]:
reset_database= "no"#@param["yes","no"]
if reset_database=="yes":
  format_database()
elif  reset_database=="no":
  pass 

database_support= "no"#@param["yes","no"]

In [ ]:
# @title ##**Question answer** { display-mode: "form" }
question_type="write"#@param["record","write"]
if database_support == "no":
  if question_type == "write":
    question=input("Enter your question : ")
    hugging_input=input_traslator(text,input_language)
    result=nlp(question=question, context=hugging_input)
    ans=output_traslator(result["answer"],input_language)
  elif question_type=="record":
    audio, sr = get_audio()
    name="question"
    name+=audio_extenstion
    scipy.io.wavfile.write(name, sr, audio)
    question=extract_question(f"./{name}")
    hugging_input=input_traslator(text,input_language)
    result=nlp(question=question, context=hugging_input)
    ans=output_traslator(result["answer"],input_language)
if database_support == "yes":   
  if question_type == "write":
    question=input("Enter your question : ")
    hugging_input=input_traslator(text,input_language)
    database(hugging_input)
    old_text=read_database()
    result=nlp(question=question, context=old_text)
    ans=output_traslator(result["answer"],input_language)
  elif question_type=="record":
    audio, sr = get_audio()
    name="question"
    name+=audio_extenstion
    scipy.io.wavfile.write(name, sr, audio)
    question=extract_question(f"./{name}")
    hugging_input=input_traslator(text,input_language)
    database(hugging_input)
    old_text=read_database()
    result=nlp(question=question, context=old_text)
    ans=output_traslator(result["answer"],input_language)

audio_fun(ans,"answer")
clear_output()  
print(f"Question : {question}")
print(f'Answer : {ans}')
Audio("./answer.mp3")

In [ ]:
# @title ##**Print database context** { display-mode: "form" }
if database_support == "yes":  
  print(old_text)
if database_support == "no":  
  print(text)  